# Phase 4: State-of-the-Art - Institutional Portfolio Management

Welcome to the final phase. Here, we implement an **Institutional Portfolio Management** system. This system scales complexity by integrating the **Black-Litterman Model** and a **Human-in-the-Loop (HITL)** branching workflow.

### System Architecture
The flow includes conditional branching and feedback loops:
```
START → Data Agent → Market Posterior (BL) → Optimizer → Human Review
                                 ↑                          |
                                 └------- [Rejected] -------┘
                                 |
                                 └------- [Approved] -------→ END
```

| Agent | Role | Logic |
|---|---|---|
| **InstitutionalDataAgent** | Fetches market equilibrium (Priors) | Sequential |
| **MarketPosteriorAgent** | Merges market data with subjective Views | Iterative |
| **PortfolioOptimizer** | Generates target weights | Sequential |
| **HumanReviewAgent** | Simulates a Portfolio Manager's approval | Branching |

## 1. Environment Setup

We use `--only-binary=:all:` to avoid C++ build errors on Windows for libraries like `matplotlib`.

In [1]:
%pip install vinagent==0.0.6.post3

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv('.env'))

llm = ChatOpenAI(model="gpt-4o-mini")
print("LLM initialized.")

Note: you may need to restart the kernel to use updated packages.
LLM initialized.


## 2. Define Institutional State

The state tracks market priors, subjective views, and the final approval status.

In [2]:
import operator
from typing import Annotated, List, TypedDict

def append_messages(existing: list, update: dict) -> list:
    return existing + [update]

class InstitutionalState(TypedDict):
    """State for professional institutional rebalancing."""
    messages: Annotated[list[dict], append_messages]
    market_priors: str
    optimal_weights: str
    approval_status: str
    pm_feedback: str

In [3]:
from datetime import datetime
from vinagent.register import primary_function

@primary_function
def get_current_time() -> str:
    """
    Get the current date and time. Use this to know 'today's' date.
    Returns:
        str: Current date and time in YYYY-MM-DD HH:MM:SS format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Custom time tool defined.")

Custom time tool defined.


## 3. Implement Institutional Agents

The `HumanReviewAgent` is the core branching node.

In [4]:
from vinagent.multi_agent import AgentNode
from vinagent.logger.logger import logging_message

class PortfolioOptimizer(AgentNode):
    """Performs mean-variance optimization using a precision Tool."""
    @logging_message
    def exec(self, state: InstitutionalState) -> dict:
        print(f"[{self.name}] Generating optimal weights...")
        returns = state.get("posterior_returns", "")
        
        prompt = f"""
        Calculate optimal weights based on: {returns}
        Use the `optimize_portfolio` tool.
        """
        output = self.invoke(prompt)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "optimal_weights": output.content if hasattr(output, "content") else str(output)
        }

## 4. Assemble the Institutional Graph

We use conditional mapping to handle the feedback loop from Human Review to Posterior adjustment.

In [5]:
# Tools imported directly
from vinagent.tools.yfinance_tools import fetch_stock_data, visualize_stock_data, plot_returns
from vinagent.tools.websearch_tools import search_api
from customize_tools import calculate_black_litterman, optimize_portfolio, calculate_equilibrium_returns

@primary_function
def get_current_time() -> str:
    """Get current system time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Institutional Core Infrastructure Initialized (Direct Registration).")


Institutional Core Infrastructure Initialized (Direct Registration).


In [6]:
from vinagent.multi_agent import AgentNode
from vinagent.logger.logger import logging_message

class InstitutionalDataAgent(AgentNode):
    """Fetches market equilibrium (Priors)."""
    @logging_message
    def exec(self, state: InstitutionalState) -> dict:
        print(f"[{self.name}] Fetching market priors...")
        prompt = """
        Calculate implied market returns (priors) for a Tech-Heavy portfolio in the Vietnam Stock Market.
        CRITICAL: You MUST use these exact Vietnamese tickers: ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'].
        Use the `calculate_equilibrium_returns` tool.
        """
        output = self.invoke(prompt)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "market_priors": output.content if hasattr(output, "content") else str(output)
        }

class MarketPosteriorAgent(AgentNode):
    """Merges market data with subjective Views using Black-Litterman."""
    @logging_message
    def exec(self, state: InstitutionalState) -> dict:
        print(f"[{self.name}] Applying Black-Litterman model...")
        priors = state.get("market_priors", "")
        feedback = state.get("pm_feedback", "No previous feedback. Assume standard positive views on tech.")
        
        prompt = f"""
        Priors: {priors}
        PM Feedback/Views: {feedback}
        
        Calculate Black-Litterman posterior returns using the `calculate_black_litterman` tool.
        Make sure to apply the views mentioned in the PM Feedback.
        """
        output = self.invoke(prompt)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "market_priors": output.content if hasattr(output, "content") else str(output) 
        }

class PortfolioOptimizer(AgentNode):
    """Performs mean-variance optimization."""
    @logging_message
    def exec(self, state: InstitutionalState) -> dict:
        print(f"[{self.name}] Generating optimal weights...")
        returns = state.get("market_priors", "")
        
        prompt = f"""
        Calculate optimal weights based on these returns: {returns}
        Use the `optimize_portfolio` tool.
        """
        output = self.invoke(prompt)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "optimal_weights": output.content if hasattr(output, "content") else str(output)
        }

class HumanReviewAgent(AgentNode):
    """Simulates a Portfolio Manager's approval."""
    @logging_message
    def exec(self, state: InstitutionalState) -> dict:
        print(f"[{self.name}] Reviewing portfolio weights...")
        weights = state.get("optimal_weights", "")
        
        prompt = f"""
        Review these proposed weights: {weights}. 
        Do they look reasonable for an institutional tech-heavy portfolio in Vietnam? 
        If the weights are sufficiently diversified (no single stock > 40%), explicitly say 'APPROVED'. 
        If they are poorly distributed or completely equal (e.g., all 0.20), explicitly say 'REJECTED' and instruct the BL agent to apply stronger subjective views (e.g., favor FPT heavily) to fix the distribution.
        """
        output = self.invoke(prompt)
        content = output.content if hasattr(output, "content") else str(output)
        
        status = "Approved" if "APPROVED" in content.upper() else "Rejected"
        
        return {
            "messages": [{"role": "assistant", "content": content}],
            "approval_status": status,
            "pm_feedback": content
        }

# 2. Instantiate the Agents
instr = 'CRITICAL: Format tool arguments as strictly valid JSON. Use double quotes (") for all property names and string values.'
no_tool_instr = "You are a Portfolio Manager evaluating output. DO NOT use tools. Just read and respond."

data_agent = InstitutionalDataAgent(name="data_agent", llm=llm, instruction=instr)
bl_agent = MarketPosteriorAgent(name="bl_agent", llm=llm, instruction=instr)
optimizer = PortfolioOptimizer(name="optimizer", llm=llm, instruction=instr)
human_review = HumanReviewAgent(name="human_review", llm=llm, instruction=no_tool_instr)

print("Phase 4 Agents defined and instantiated successfully!")

Phase 4 Agents defined and instantiated successfully!


In [7]:
global_tools = [
    primary_function(fetch_stock_data),
    primary_function(visualize_stock_data),
    primary_function(plot_returns),
    primary_function(search_api),
    primary_function(calculate_black_litterman),
    primary_function(optimize_portfolio),
    primary_function(calculate_equilibrium_returns),
    get_current_time
]

for agent in [data_agent, bl_agent, optimizer, human_review]:
    for tool in global_tools:
        agent.tools_manager.register_function_tool(tool)

from vinagent.multi_agent import CrewAgent
from vinagent.graph.operator import FlowStateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

def human_review_router(state):
    """Routes back to the Black-Litterman agent if rejected, or ends if approved."""
    if state.get("approval_status") == "Approved":
        return END
    return bl_agent.name

institutional_graph = FlowStateGraph(InstitutionalState)
institutional_graph.add_conditional_edges(human_review.name, human_review_router)

crew = CrewAgent(
    llm=llm,
    checkpoint=MemorySaver(),
    graph=institutional_graph,
    flow=[
        START >> data_agent,
        data_agent >> bl_agent,
        bl_agent >> optimizer,
        optimizer >> human_review
    ]
)

print("Phase 4 HITL Crew Assembled successfully!")

INFO:vinagent.register.tool:Registered tool: fetch_stock_data (runtime)
INFO:vinagent.register.tool:Registered tool: visualize_stock_data (runtime)
INFO:vinagent.register.tool:Registered tool: plot_returns (runtime)
INFO:vinagent.register.tool:Registered tool: search_api (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_black_litterman (runtime)
INFO:vinagent.register.tool:Registered tool: optimize_portfolio (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_equilibrium_returns (runtime)
INFO:vinagent.register.tool:Registered tool: get_current_time (runtime)
INFO:vinagent.register.tool:Registered tool: fetch_stock_data (runtime)
INFO:vinagent.register.tool:Registered tool: visualize_stock_data (runtime)
INFO:vinagent.register.tool:Registered tool: plot_returns (runtime)
INFO:vinagent.register.tool:Registered tool: search_api (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_black_litterman (runtime)
INFO:vinagent.register.tool:Registered tool

Phase 4 HITL Crew Assembled successfully!


## 5. Execute Institutional Rebalance

The system will iterate until the Human Review node grants approval.

In [8]:
from IPython.display import display, Markdown

query = "Perform an institutional rebalance for the Tech-Heavy portfolio in Vietnam Stock Market. Ensure it goes through PM review."

result = crew.invoke(query=query, user_id="admin", thread_id=10)

display(Markdown(f"""
## 🏦 Institutional Portfolio Advice
---
**Approval Status:** {result.get('approval_status', 'Unknown')}

**Optimized Allocation Insights:**
{result.get('optimal_weights', 'No data collected.')}
"""))

INFO:vinagent.multi_agent.crew:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:vinagent.agent.agent:Tool calling iteration 1/10


{'input': {'messages': {'role': 'user', 'content': 'Perform an institutional rebalance for the Tech-Heavy portfolio in Vietnam Stock Market. Ensure it goes through PM review.'}}, 'config': {'configurable': {'user_id': 'admin'}, 'thread_id': 10}}
[data_agent] Fetching market priors...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'calculate_equilibrium_returns', 'tool_type': 'function', 'arguments': {'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'risk_free_rate': 0.03, 'market_risk_premium': 0.07}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool calculate_equilibrium_returns({'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'risk_free_rate': 0.03, 'market_risk_premium': 0.07})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ InstitutionalDataAgent Response ============
{'messages': [{'role': 'assistant', 'content': "The implied market returns for a Tech-Heavy portfolio in the Vietnam Stock Market using 

[bl_agent] Applying Black-Litterman model...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'calculate_black_litterman', 'tool_type': 'function', 'arguments': {'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'priors': [10.7, 10.0, 10.0, 10.0, 10.0], 'views': [], 'tau': 0.05}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool calculate_black_litterman({'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'priors': [10.7, 10.0, 10.0, 10.0, 10.0], 'views': [], 'tau': 0.05})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ MarketPosteriorAgent Response ============
{'messages': [{'role': 'assistant', 'content': 'The Black-Litterman posterior returns for the Tech-Heavy portfolio i

[optimizer] Generating optimal weights...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'optimize_portfolio', 'tool_type': 'function', 'arguments': {'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'expected_returns': {'FPT': 0.107, 'CMG': 0.1, 'VGI': 0.1, 'ELC': 0.1, 'ITD': 0.1}, 'risk_aversion': 2.0}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool optimize_portfolio({'symbols': ['FPT', 'CMG', 'VGI', 'ELC', 'ITD'], 'expected_returns': {'FPT': 0.107, 'CMG': 0.1, 'VGI': 0.1, 'ELC': 0.1, 'ITD': 0.1}, 'risk_aversion': 2.0})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ PortfolioOptimizer Response ============
{'messages': [{'role': 'assistant', 'content': 'The opti

[human_review] Reviewing portfolio weights...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 1 iterations.
INFO:vinagent.logger.logger:

============ HumanReviewAgent Response ============
{'messages': [{'role': 'assistant', 'content': 'APPROVED'}], 'approval_status': 'Approved', 'pm_feedback': 'APPROVED'}





## 🏦 Institutional Portfolio Advice
---
**Approval Status:** Approved

**Optimized Allocation Insights:**
The optimal weights for the Tech-Heavy portfolio in the Vietnam Stock Market based on the Black-Litterman posterior returns are as follows:

- FPT: 21.1%
- CMG: 19.72%
- VGI: 19.72%
- ELC: 19.72%
- ITD: 19.72%
